In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
# display all columns
pd.options.display.max_columns = None

/Users/nguyenquangminh/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Utils
from sklearn import preprocessing
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None, test: bool = False, test_dataset = None):
    if test == True:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        test_dataset[name] = (test_dataset[name] - mean) / sd
    else:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        df[name] = (df[name] - mean) / sd

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

In [3]:
path = "/Users/nguyenquangminh/Desktop/Thesis/Dataset/CICDataset/CIC-IDS-2017/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv"
custom_dataset_path = "../../Dataset/syn-flood2.pcap_Flow.csv"
dataset = pd.read_csv(filepath_or_buffer=path)
custom_dataset = pd.read_csv(filepath_or_buffer=custom_dataset_path)
dataset

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.14-209.48.71.168-49459-80-6,192.168.10.14,49459,209.48.71.168,80,6,5/7/2017 8:42,38308,1,1,6,6.0,6,6,6.000000,0.000000,6,6,6.000000,0.000000,3.132505e+02,52.208416,38308.000000,0.000000,38308.0,38308.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0,0,0,0,20,20,26.104208,26.104208,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,9.000000,6.000000,6.000000,20,0,0,0,0,0,0,1,6,1,6,255,946,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.3-192.168.10.17-389-49453-6,192.168.10.17,49453,192.168.10.3,389,6,5/7/2017 8:42,479,11,5,172,326.0,79,0,15.636364,31.449238,163,0,65.200000,89.278777,1.039666e+06,33402.922760,31.933333,25.510409,73.0,0.0,479.0,47.900000,38.942836,109.0,1.0,401.0,100.250000,101.736178,237.0,3.0,0,0,0,0,368,176,22964.509390,10438.413360,0,163,29.294118,56.529599,3195.595588,0,0,0,1,0,0,0,0,0,31.125000,15.636364,65.200000,368,0,0,0,0,0,0,11,172,5,326,29200,260,4,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.17-88-46124-6,192.168.10.17,46124,192.168.10.3,88,6,5/7/2017 8:42,1095,10,6,3150,3150.0,1575,0,315.000000,632.561635,1575,0,525.000000,813.326503,5.753425e+06,14611.872150,73.000000,204.960972,810.0,1.0,1095.0,121.666667,298.746130,915.0,1.0,995.0,199.000000,345.535092,810.0,3.0,0,0,0,0,336,208,9132.420091,5479.452055,0,1575,370.588235,671.751541,451250.132400,0,0,0,1,0,0,0,0,0,393.750000,315.000000,525.000000,336,0,0,0,0,0,0,10,3150,6,3150,29200,2081,3,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.17-389-49454-6,192.168.10.17,49454,192.168.10.3,389,6,5/7/2017 8:42,15206,17,12,3452,6660.0,1313,0,203.058823,425.778474,3069,0,555.000000,977.480342,6.650007e+05,1907.141918,543.071429,2519.931377,13391.0,0.0,15206.0,950.375000,3322.417812,13391.0,2.0,15112.0,1373.818182,4176.449588,13961.0,3.0,0,0,0,0,560,388,1117.979745,789.162173,0,3069,337.066667,704.654082,496537.374700,0,0,0,1,0,0,0,0,0,348.689655,203.058823,555.000000,560,0,0,0,0,0,0,17,3452,12,6660,29200,0,10,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.17-88-46126-6,192.168.10.17,46126,192.168.10.3,88,6,5/7/2017 8:42,1092,9,6,3150,3152.0,1575,0,350.000000,694.509719,1576,0,525.333333,813.842901,5.771062e+06,13736.263740,78.000000,207.000929,794.0,1.0,1092.0,136.500000,313.850738,910.0,1.0,1015.0,203.000000,333.240154,794.0,3.0,0,0,0,0,304,208,8241.758242,5494.505495,0,1576,393.875000,704.585067,496440.116700,0,0,0,1,0,0,0,0,0,420.133333,350.000000,525.333333,304,0,0,0,0,0,0,9,3150,6,3152,29200,2081,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [4]:
dataset.drop(columns=[' Fwd Header Length.1'], inplace=True)
custom_dataset.columns = dataset.columns
custom_dataset[" Label"] = "BENIGN"

dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)
custom_dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)

In [5]:
for column in custom_dataset.columns:
    if column != " Label":
        encode_numeric_zscore(df=dataset, name=column, test=True, test_dataset=custom_dataset)
    else:
        print(encode_text_index(custom_dataset, column))
        
for column in dataset.columns:
    if column != " Label":
        encode_numeric_zscore(dataset, column)
    else:
        print(encode_text_index(dataset, column))

['BENIGN']
['BENIGN' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest' 'DoS slowloris'
 'Heartbleed']


In [6]:
dataset

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.356503,-0.642646,-0.653857,-0.011451,-0.009362,-0.089086,-0.007581,-0.376966,-0.176667,-0.346068,-0.366591,-0.633357,-0.430940,-0.684609,-0.599827,NaN,NaN,-0.440408,-0.582296,-0.595270,-0.050133,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.014196,-0.011643,-0.298339,-0.130224,-0.280643,-0.652576,-0.737535,-0.667906,-0.507522,-0.328313,-0.209906,-0.018383,-0.490125,1.179856,3.750547,NaN,-0.018422,0.776269,-0.745303,-0.346068,-0.684609,NaN,NaN,NaN,NaN,NaN,NaN,-0.011451,-0.089086,-0.009362,-0.007581,-0.425003,-0.072539,-0.008559,-1.069400,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
1,-0.336856,-0.642646,-0.654742,0.001932,-0.005298,-0.062153,-0.007438,-0.256055,-0.294156,-0.284940,-0.227513,-0.573294,-0.523839,-0.610372,-0.518520,NaN,NaN,-0.447248,-0.582294,-0.596266,-0.060562,-0.648671,-0.459331,-0.563338,-0.593144,-0.116252,-0.415666,-0.275905,-0.328058,-0.355611,-0.114314,-0.209906,NaN,NaN,NaN,0.008051,-0.003720,-0.226625,0.206534,-0.498652,-0.592980,-0.674429,-0.595750,-0.505694,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,-0.689719,-0.284940,-0.610372,NaN,NaN,NaN,NaN,NaN,NaN,0.001932,-0.062153,-0.005298,-0.007438,2.013552,-0.166315,-0.002966,0.828623,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
2,-0.355994,-0.642646,-0.654727,0.000594,-0.004282,0.421001,-0.006178,2.221784,-0.294156,1.614047,2.430793,-0.033110,-0.523839,-0.033784,0.140878,NaN,NaN,-0.447241,-0.582279,-0.596247,-0.060562,-0.648657,-0.459325,-0.563322,-0.593123,-0.116252,-0.415648,-0.275895,-0.328036,-0.355590,-0.114314,-0.209906,NaN,NaN,NaN,0.006005,-0.002095,-0.269869,0.046149,-0.498652,-0.056989,0.250160,0.189530,-0.249501,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,0.221294,1.614047,-0.033784,NaN,NaN,NaN,NaN,NaN,NaN,0.000594,0.421001,-0.004282,-0.006178,2.013552,0.082616,-0.004364,0.828623,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
3,-0.336856,-0.642646,-0.654397,0.009962,0.001815,0.469998,-0.004612,1.787831,-0.294156,0.903958,1.516334,0.538445,-0.523839,0.003836,0.290375,NaN,NaN,-0.447157,-0.582082,-0.595919,-0.060562,-0.648327,-0.459250,-0.563133,-0.592799,-0.116252,-0.415224,-0.275772,-0.327679,-0.355087,-0.114314,-0.209906,NaN,NaN,NaN,0.020325,0.007047,-0.294925,-0.105545,-0.498652,0.510129,0.159348,0.231527,-0.223607,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,0.108090,0.903958,0.003836,NaN,NaN,NaN,NaN,NaN,NaN,0.009962,0.469998,0.001815,-0.004612,2.013552,-0.201858,0.005424,0.82

In [7]:
dataset[" Label"].value_counts()

0    440031
2    231073
1     10293
4      5796
3      5499
5        11
Name:  Label, dtype: int64

In [8]:
custom_dataset

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.123216,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447250,-0.582296,-0.596267,-0.060555,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.014196,-0.011643,-0.173366,1.162623,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,-0.360216,-0.201858,-0.008559,-1.069400,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
1,-0.008385,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447249,-0.582296,-0.596267,-0.060555,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.014196,-0.011643,-0.178176,1.112866,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,5.074698,-0.201858,-0.008559,-1.069400,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
2,0.175244,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447249,-0.582296,-0.596267,-0.060555,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.014196,-0.011643,-0.178176,1.112866,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,-0.360216,-0.201858,-0.008559,-1.069400,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
3,0.422137,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447248,-0.582296,-0.596267,-0.060553,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.013940,-0.011643,-0.206469,0.820175,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,-0

In [9]:
custom_dataset[" Label"].value_counts()

0    207
Name:  Label, dtype: int64

In [10]:
nan_columns = dataset.columns[dataset.isna().any()]
print(nan_columns)
test_nan_columns = custom_dataset.columns[custom_dataset.isna().any()]
print(test_nan_columns)
dataset.drop(columns=nan_columns, inplace=True)
custom_dataset.drop(columns=nan_columns, inplace=True)

Index(['Flow Bytes/s', ' Flow Packets/s', ' Bwd PSH Flags', ' Fwd URG Flags',
       ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')
Index(['Flow Bytes/s', ' Flow Packets/s', ' Bwd PSH Flags', ' Fwd URG Flags',
       ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')


In [11]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(dataset,' Label')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
16236/16236 - 13s - loss: 0.1334 - val_loss: 0.0942 - 13s/epoch - 774us/step
Epoch 2/1000
16236/16236 - 14s - loss: 0.0953 - val_loss: 0.0894 - 14s/epoch - 836us/step
Epoch 3/1000
16236/16236 - 14s - loss: 0.0896 - val_loss: 0.0821 - 14s/epoch - 841us/step
Epoch 4/1000
16236/16236 - 23s - loss: 0.0873 - val_loss: 0.0822 - 23s/epoch - 1ms/step
Epoch 5/1000
16236/16236 - 19s - loss: 0.0860 - val_loss: 0.0821 - 19s/epoch - 1ms/step
Epoch 6/1000
16236/16236 - 18s - loss: 0.0835 - val_loss: 0.0809 - 18s/epoch - 1ms/step
Epoch 7/1000
16236/16236 - 17s - loss: 0.0837 - val_loss: 0.0847 - 17s/epoch - 1ms/step
Epoch 8/1000
16236/16236 - 17s - loss: 0.0830 - val_loss: 0.1151 - 17s/epoch - 1ms/step
Epoch 9/1000
16236/16236 - 17s - loss: 0.0829 - val_loss: 0.0881 - 17s/epoch - 1ms/step
Epoch 10/1000
16236/16236 - 21s - loss: 0.0794 - val_loss: 0.0827 - 21s/epoch - 1ms/step
Epoch 11/1000
16236/16236 - 20s - loss: 0.0795 - val_loss: 0.0799 - 20s/epoch - 1ms/step
Epoch 12/1000
16236/1623

In [12]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

5412/5412 [==============================] - 2s 303us/step
Validation score: 0.9803090497528526


In [13]:
x_custom, y_custom = to_xy(custom_dataset, " Label")

In [14]:
# Measure accuracy
pred = model.predict(x_custom)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_custom,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation sore: {}".format(score))

7/7 [==============================] - 0s 452us/step
Validation sore: 0.9710144927536232


In [15]:
#96 - 99%